In [1]:
import os
import tensorflow as tf
from Utilities.configuration import configuration, segmentation_index
from Utilities.utilities import CNNUtilities
from NNFactory.NNFactoryVGG import NNFactoryWithVGG

2023-06-13 08:53:15.631209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 08:53:16.233147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(f'[CONFIGURATION] Actual configuration {configuration}')

if not configuration['GPU']:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
else:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    for i in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[i], True)

[CONFIGURATION] Actual configuration {'VERBOSE': False, 'GPU': False, 'image_height': 512, 'image_width': 928, 'epochs': 200}


In [6]:
# create utility instance
utilities = CNNUtilities(configuration=configuration,
                         segmentation_index_list=segmentation_index)

# get height and width of the image
h = configuration["image_height"]
w = configuration["image_width"]

# define the var for the final shape
image_shape = (h, w)

In [7]:
# first step load the datasets
train_rgb, train_depth, train_labels = utilities.load_the_image_from_the_dataset_folder(path="../Dataset/train",
                                                                                        image_shape=image_shape)

print(f"[TRAIN] RGB {train_rgb.shape} NIR {train_depth.shape} LABEL {train_labels.shape}")

test_rgb, test_depth, test_labels = utilities.load_the_image_from_the_dataset_folder(path="../Dataset/test",
                                                                                     image_shape=image_shape)

print(f"[TEST] RGB {test_rgb.shape} NIR {test_depth.shape} LABEL {test_labels.shape}")

validation_rgb, validation_depth, validation_labels = utilities.load_the_image_from_the_dataset_folder(path="../Dataset/valid",
                                                                                                       image_shape=image_shape)

print(f"[VALIDATION] RGB {validation_rgb.shape} NIR {validation_depth.shape} LABEL {validation_labels.shape}")

[TRAIN] RGB (200, 512, 928, 3) NIR (200, 512, 928, 3) LABEL (200, 475136, 6)
[TEST] RGB (100, 512, 928, 3) NIR (100, 512, 928, 3) LABEL (100, 475136, 6)
[VALIDATION] RGB (66, 512, 928, 3) NIR (66, 512, 928, 3) LABEL (66, 475136, 6)


In [8]:
# create cnn config
cnn_configuration = {
    'shape_stream_rgb': train_rgb.shape[1:],
    'kernel_size_stream_rgb': (train_rgb.shape[3], train_rgb.shape[3]),
    'shape_stream_nir': train_depth.shape[1:],
    'kernel_size_stream_nir': (train_depth.shape[3], train_depth.shape[3]),
    'list_of_conv_layers': [128, 256],
    "dropout": 0.2,
    'number_of_classes': len(segmentation_index)
    }

print(f'[CNN CONFIGURATION] {cnn_configuration}')

In [9]:
# prepare dicts of data
# for train
dictionary_of_training = {
    'input_rgb': train_rgb,
    'input_nir': train_depth
}
# for test
dictionary_of_test = {
    'input_rgb': test_rgb,
    'input_nir': test_depth
}
# for validation
dictionary_of_validation = {
    'input_rgb': validation_rgb,
    'input_nir': validation_depth
}

In [10]:
# instance of the model
cnn_handler = NNFactoryWithVGG(**cnn_configuration)

# create a late fusion
cnn_handler.middle_fusion()

/home/luca/anaconda3/envs/fusion2023/lib/python3.11/site-packages/keras/applications/vgg16.py:137: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 6 input channels.
  input_shape = imagenet_utils.obtain_input_shape(
2023-06-13 09:03:26.035624: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-13 09:03:26.035693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: luca-arlecchino
2023-06-13 09:03:26.035710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: luca-arlecchino
2023-06-13 09:03:26.035843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.105.17
2023-06-13 09:03:26.035898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported vers

In [ ]:
cnn_handler.plot_the_model("cnn_middle_fusion.png")

In [ ]:
# fit the model
cnn_handler.fit_the_model(x_train=dictionary_of_training,
                          y_train=train_labels,
                          x_validation=dictionary_of_validation,
                          y_validation=validation_labels,
                          epochs=configuration["epochs"])


In [ ]:
# evaluate the model
cnn_handler.evaluate_the_model(x_test=dictionary_of_test,
                               y_test=test_labels)


In [ ]:
# predict
final_predictions = cnn_handler.make_predictions(x_test=dictionary_of_test)


In [ ]:
# loop into the prediction arrays and rebuilt the image
for predicted_index in range(final_predictions.shape[0]):
    temp = utilities.convert_the_prediction(prediction=final_predictions[predicted_index, :, :],
                                            number_of_classes=len(segmentation_index),
                                            height_of_predicted_image=h,
                                            width_of_pred_image=w)
